<a href="https://colab.research.google.com/github/JTenaSan/CS3120_EDA_Jesus-Tena/blob/main/Spotify_EDA_Modeling_Jesus_Tena.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.ticker import ScalarFormatter
import kagglehub

# Download latest version
path = kagglehub.dataset_download("irynatokarchuk/top-streamed-spotify-songs-by-year-2010-2023")
print("Path to dataset files:", path)

# Check if the downloaded folder exists
if os.path.exists(path):
    # List the files in the downloaded folder
    files = os.listdir(path)
    print("Files in the downloaded folder:", files)

    # Find the CSV file
    csv_file = next((f for f in files if f.endswith('.csv')), None)

    if csv_file:
        file_path = os.path.join(path, csv_file)  # Use the found CSV file
        df = pd.read_csv(file_path)
    else:
        print("CSV file not found in the downloaded folder.")
else:
    print("Downloaded folder not found.")

100%|██████████| 425k/425k [00:00<00:00, 72.0MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/irynatokarchuk/top-streamed-spotify-songs-by-year-2010-2023/versions/1
Files in the downloaded folder: ['spotify_full_list_20102023.csv']
